In [56]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from numba import jit, njit, cuda
import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import scipy.signal as ss
import scipy.integrate as si
import scipy.optimize

import time

mctrajlist = []
for i in range(100):
    mctrajlist.append(np.genfromtxt('./mcsinnew/mcsinnew' + str(i+1) + '.csv', delimiter=','))
    # mctrajlist.append(np.genfromtxt('/pscratch/sd/h/hbhat512/mcdblwell1d/mcdblwell1d' + str(i+1) + '.csv', delimiter=','))

mctraj = np.stack(mctrajlist, 1)
print(mctraj.shape)

dilfac = 100
mctraj = mctraj[::dilfac,:]
#plt.plot(mctraj)
#plt.show()
numtraj = mctraj.shape[1]
lentraj = mctraj.shape[0]
print(mctraj.shape)

(4001, 100)
(41, 100)


In [93]:
# time step of the data
deltat = 0.001*dilfac

# crucial parameter that sets internal time step
numsteps = dilfac
h = deltat/numsteps

# constant diffusion
g = 0.25

# levy alpha parameter
alpha = 1.0

# take L = 2
bigL = 2

# set up drift f(x) = sin(x)
bigJ = 4
modes = np.arange(-bigJ,bigJ+1)/bigL

# truetheta = np.zeros((2*bigJ+1), dtype=np.complex128)
# for j in range(-bigJ, bigJ+1):
#     if j != 0:
#         truetheta[j] = -2j*(-1)**j*(-24 + j**2 * (-1 + 4*np.pi**2))/j**3

truetheta = np.zeros((2*bigJ+1), dtype=np.complex128)
truetheta[bigJ+bigL] = -0.5j
truetheta[bigJ-bigL] = 0.5j

# set n2, number of points to the left of the origin
n2 = 1024
npts = 2*n2 + 1

# take n_L = 2
nL = 8
ds = 1.0/(bigL*nL)
uvec = np.arange(-n2,n2+1)*ds
umin = np.amin(uvec)
umax = np.amax(uvec)

# initial char fun
psi0 = np.sum(np.exp(1j*np.einsum('i,jk->ijk',uvec,mctraj[:-1,:])),2)/numtraj
print(psi0.shape)

# target char fun
psitarget = np.sum(np.exp(1j*np.einsum('i,jk->ijk',uvec,mctraj[1:,:])),2)/numtraj
print(psitarget.shape)

# need this guy to propagate
diagfac = np.exp(-h*np.abs(uvec*g)**alpha)

(2049, 40)
(2049, 40)


In [60]:
print(ds)

0.0625


In [61]:
@cuda.jit('void(complex128[:,:], complex128[:,:], complex128[:], complex128[:], float64[:])')
def update(newpsi, oldpsi, dtheta, dtheta2, ddiagfac):
    j, m = cuda.grid(2)
    if j < npts and m < (lentraj-1):
        term1 = 0.0
        # j is the Python index, truej is the Math index
        truej = j - n2
        # jp is the Python index, "truejp"=jp-bigJ is the Math index
        for jp in range(2*bigJ+1):
            # suppose |truej + truejp*nL| <= n2, then 0 <= j + truejp*nL <= 2*n2,
            # enabling us to access oldpsi at location "j + truejp*nL":
            if abs(truej + (jp-bigJ)*nL) <= n2:
                term1 += dtheta[jp]*oldpsi[j + (jp-bigJ)*nL,m]
        newpsi[j,m] += 1j*(h*ds)*truej*term1
        term2 = 0.0
        # k is the Python index, "truek"=k-2*bigJ is the Math index
        for k in range(4*bigJ+1):
            # suppose |truej + truek*nL| <= n2, then 0 <= j + truek*nL <= 2*n2,
            # enabling us to access oldpsi at location "j + truek*nL":
            if abs(truej + (k-2*bigJ)*nL) <= n2:
                term2 += dtheta2[k]*oldpsi[j + (k-2*bigJ)*nL,m]
        newpsi[j,m] += -0.5*((h*ds)**2)*(truej**2)*term2
        newpsi[j,m] *= ddiagfac[j]

@cuda.jit
def copyover(newpsi, oldpsi):
    j, k = cuda.grid(2)
    if j < npts and k < (lentraj-1):
        oldpsi[j,k] = newpsi[j,k]


In [62]:
@cuda.jit('void(complex128[:,:], complex128[:,:], complex128[:], complex128[:], float64[:])')
def updatev2(newpsi, oldpsi, dtheta, dtheta2, ddiagfac):
    j, m = cuda.grid(2)
    if j < npts and m < (lentraj-1):
        newpsi[j,m] = oldpsi[j,m]
        term1 = 0.0
        # j is the Python index, truej is the Math index
        truej = j - n2
        # jp is the Python index, "truejp"=jp-bigJ is the Math index
        for jp in range(2*bigJ+1):
            # suppose |truej + truejp*nL| <= n2, then 0 <= j + truejp*nL <= 2*n2,
            # enabling us to access oldpsi at location "j + truejp*nL":
            if abs(truej + (jp-bigJ)*nL) <= n2:
                term1 += dtheta[jp]*oldpsi[j + (jp-bigJ)*nL,m]
        newpsi[j,m] += 1j*(h*ds)*truej*term1
        term2 = 0.0
        # k is the Python index, "truek"=k-2*bigJ is the Math index
        for k in range(4*bigJ+1):
            # suppose |truej + truek*nL| <= n2, then 0 <= j + truek*nL <= 2*n2,
            # enabling us to access oldpsi at location "j + truek*nL":
            if abs(truej + (k-2*bigJ)*nL) <= n2:
                term2 += dtheta2[k]*oldpsi[j + (k-2*bigJ)*nL,m]
        newpsi[j,m] += -0.5*((h*ds)**2)*(truej**2)*term2
        newpsi[j,m] *= ddiagfac[j]


In [63]:
@njit
def newCPUupdate(theta, psi0):
    psi1 = np.zeros(npts, dtype=np.complex128)
    theta2 = np.zeros(4*bigJ+1, dtype=np.complex128)
    for k in range(4*bigJ+1):
        for j in range(2*bigJ+1):
            if ((k-j) >= 0) and ((k-j) <= 2*bigJ):
                theta2[k] += theta[j]*theta[k-j]
    for j in range(npts):
        term1 = 0.0
        truej = j - n2
        psi1[j] = psi0[j]
        for jp in range(2*bigJ+1):
            if abs(truej + (jp-bigJ)*nL) <= n2:
                tmp = psi0[j + (jp-bigJ)*nL]
                term1 += theta[jp]*tmp
        
        term2 = 0.0
        for k in range(4*bigJ+1):
            if abs(truej + (k-2*bigJ)*nL) <= n2:
                tmp = psi0[j + (k-2*bigJ)*nL]
                term2 += theta2[k]*tmp
        
        psi1[j] += 1j*(h*ds)*term1*truej
        psi1[j] += -0.5*((h*ds)**2)*term2*truej**2
        psi1[j] *= diagfac[j]
    return psi1


In [64]:
# propagate for 5 steps
# see if you can get derivative of this loss w.r.t. theta!
@njit
def MMDloss(theta):
    loss = 0.0
    ntraj = psi0.shape[1]
    for traj in range(ntraj):
        psicur = np.copy(psi0[:,traj])
        for j in range(numsteps):
            psicur = newCPUupdate(theta, psicur)
        loss += 0.5*np.sum(np.square(np.abs(psicur - psitarget[:,traj])))
    return loss/ntraj

In [65]:
# MMDloss(truetheta)

In [66]:
ddiagfac = cuda.to_device(diagfac)

def MMDlossGPU(theta):
    theta2 = ss.convolve(theta, theta)
    dtheta = cuda.to_device(theta)
    dtheta2 = cuda.to_device(theta2)
    
    # do the entire trajectory at once
    psi0 = np.sum(np.exp(1j*np.einsum('i,jk->ijk',uvec,mctraj[:-1,:])),2)/numtraj
    doldpsi = cuda.to_device(psi0)
    dnewpsi = cuda.to_device(psi0)

    threadsperblock = (8,10) # try 8, 10, 10
    blockspergrid_x = int(np.ceil(psi0.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(psi0.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    for k in range(numsteps):
        update[blockspergrid, threadsperblock](dnewpsi, doldpsi, dtheta, dtheta2, ddiagfac)
        copyover[blockspergrid, threadsperblock](dnewpsi, doldpsi)
    
    psifinal = dnewpsi.copy_to_host()
    psitarget = np.sum(np.exp(1j*np.einsum('i,jk->ijk',uvec,mctraj[1:,:])),2)/numtraj
    
    return 0.5*np.sum(np.square(np.abs(psifinal - psitarget)))/(lentraj-1)

In [67]:
# MMDlossGPU(truetheta)

In [68]:
@cuda.jit('void(complex128[:,:,:], complex128[:,:], complex128[:], float64[:])')
def GPUderivupdate(psi1, psi0, theta, diagfac):
    j, m = cuda.grid(2)
    if j < npts and m < (lentraj-1):
        truej = j - n2
        for r in range(2*bigJ+1):
            psi1[j,r,m] = 0.0
            truer = r - bigJ
            if abs(truej + truer*nL) <= n2:
                tmp = psi0[j + truer*nL,m]
                psi1[j,r,m] += 1j*(h*ds)*truej*diagfac[j]*tmp

            for k in range(4*bigJ+1):
                truek = k - 2*bigJ
                if abs(truek-truer) <= bigJ:
                    if abs(truej + truek*nL) <= n2:
                        tmp = psi0[j + truek*nL,m]
                        term2 = theta[truek-truer+bigJ]
                        psi1[j,r,m] += -((h*ds)**2)*(truej**2)*term2*tmp*diagfac[j]
    
# let us assume that oldlamb is of size (npts, npts)
# we need to compute newlamb of the same size

@cuda.jit('void(complex128[:,:], complex128[:,:], complex128[:], complex128[:], float64[:])')
def GPUlambda(newlamb, oldlamb, theta, theta2, diagfac):
    l, m = cuda.grid(2)
    if l < npts and m < (lentraj-1):
        newlamb[l,m] = oldlamb[l,m]*diagfac[l]
        truel = l - n2
        for k in range(2*bigJ):
            truek = k - bigJ
            ind = truel - truek*nL
            if abs(ind)<=n2:
                dd = diagfac[ind+n2]
                newlamb[l,m] += dd*1j*(h*ds)*oldlamb[ind+n2,m]*ind*theta[k]
        for k in range(4*bigJ):
            truek = k - 2*bigJ
            ind = truel - truek*nL
            if abs(ind)<=n2:
                term2 = theta2[k]
                dd = diagfac[ind+n2]
                newlamb[l,m] -= dd*0.5*(h*ds)**2*oldlamb[ind+n2,m]*term2*ind**2

# adjoint method
def MMDadjGPU(theta):
    theta2 = ss.convolve(theta, theta)
    dtheta = cuda.to_device(theta)
    dtheta2 = cuda.to_device(theta2)
    
    # do the entire trajectory at once
    # everybody
    psimat = np.zeros((numsteps+1, npts, lentraj-1), dtype=np.complex128)
    psimat[0,:,:] = np.sum(np.exp(1j*np.einsum('i,jk->ijk',uvec,mctraj[:-1,:])),2)/numtraj
    dpsimat = cuda.to_device(psimat)

    threadsperblock = (8,10) # try 8, 10, 10
    blockspergrid_x = int(np.ceil(psi0.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(psi0.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    for k in range(numsteps):
        updatev2[blockspergrid, threadsperblock](dpsimat[k+1,:,:], dpsimat[k,:,:], dtheta, dtheta2, ddiagfac)
    
    resid = dpsimat[-1,:,:].copy_to_host() - psitarget
    loss = 0.5*np.sum(np.square(np.abs(resid)))/(lentraj-1)
    # everybody
    lambmat = np.zeros((numsteps+1, npts, lentraj-1), dtype=np.complex128)
    lambmat[numsteps,:,:] = resid
    dlambmat = cuda.to_device(lambmat)
    
    for j in range(numsteps,0,-1):
        GPUlambda[blockspergrid, threadsperblock](dlambmat[j-1,:,:], dlambmat[j,:,:], dtheta, dtheta2, ddiagfac)
    
    # now form the derivative
    lambmatconj = dlambmat.copy_to_host().conj()
    derivmat = np.zeros(2*bigJ+1, dtype=np.complex128)
    ddu = cuda.to_device(np.zeros((npts, 2*bigJ+1, lentraj-1), dtype=np.complex128))
    for j in range(numsteps):
        GPUderivupdate[blockspergrid, threadsperblock](ddu, dpsimat[j,:,:], dtheta, ddiagfac)
        du = ddu.copy_to_host()
        derivmat += np.einsum('ik,ijk->j',lambmatconj[j+1,:,:],du)/(lentraj-1)
        
    return loss, derivmat

In [69]:
@njit
def derivupdate(theta, psi0):
    psi1 = np.zeros((npts,2*bigJ+1), dtype=np.complex128)
    for j in range(npts):
        truej = j - n2
        for r in range(2*bigJ+1):
            truer = r - bigJ
            if abs(truej + truer*nL) <= n2:
                tmp = psi0[j + truer*nL]
                psi1[j,r] += 1j*(h*ds)*truej*diagfac[j]*tmp

            for k in range(4*bigJ+1):
                truek = k - 2*bigJ
                if abs(truek-truer) <= bigJ:
                    if abs(truej + truek*nL) <= n2:
                        tmp = psi0[j + truek*nL]
                        term2 = theta[truek-truer+bigJ]
                        psi1[j,r] += -((h*ds)**2)*(truej**2)*term2*tmp*diagfac[j]
    return psi1
    
# let us assume that oldlamb is of size (npts, npts)
# we need to compute newlamb of the same size

@njit
def CPUlambda(theta, oldlamb):
    newlamb = np.zeros(npts, dtype=np.complex128)
    theta2 = np.zeros(4*bigJ+1, dtype=np.complex128)
    for k in range(4*bigJ+1):
        for j in range(2*bigJ+1):
            if ((k-j) >= 0) and ((k-j) <= 2*bigJ):
                theta2[k] += theta[j]*theta[k-j]
    for l in range(npts):
        newlamb[l] = oldlamb[l]*diagfac[l]
        truel = l - n2
        for k in range(2*bigJ):
            truek = k - bigJ
            ind = truel - truek*nL
            if abs(ind)<=n2:
                dd = diagfac[ind+n2]
                newlamb[l] += dd*1j*(h*ds)*oldlamb[ind+n2]*ind*theta[k]
        for k in range(4*bigJ):
            truek = k - 2*bigJ
            ind = truel - truek*nL
            if abs(ind)<=n2:
                term2 = theta2[k]
                dd = diagfac[ind+n2]
                newlamb[l] -= dd*0.5*(h*ds)**2*oldlamb[ind+n2]*term2*ind**2
    return newlamb

# adjoint method
@njit
def MMDadj(theta):
    ntraj = psi0.shape[1]
    loss = 0.0
    derivmat = np.zeros(2*bigJ+1, dtype=np.complex128)
    for traj in range(ntraj):
        # forward prop
        psimat = np.zeros((numsteps+1, npts), dtype=np.complex128)
        psimat[0, :] = psi0[:,traj]
        for j in range(numsteps):
            psimat[j+1, :] = newCPUupdate(theta, psimat[j, :])

        resid = psimat[-1,:] - psitarget[:,traj]
        loss += 0.5*np.sum(np.square(np.abs(resid)))/ntraj
        # reverse prop
        lambmat = np.zeros((numsteps+1, npts), dtype=np.complex128)
        lambmat[numsteps, :] = resid
        for j in range(numsteps,0,-1):
            lambmat[j-1, :] = CPUlambda(theta, lambmat[j, :])
        # now form the derivative
        for j in range(numsteps):
            du = derivupdate(theta, psimat[j,:])
            derivmat += (lambmat[j+1,:].conj() @ du)/ntraj
    return loss, derivmat

In [70]:
# start = time.time()
# cpuloss, cpuderivmat = MMDadj(truetheta)
# end = time.time()
# print(end-start)

In [71]:
# start = time.time()
# gpuloss, gpuderivmat = MMDadjGPU(truetheta)
# end = time.time()
# print(end-start)

In [72]:
# # ii = 99
# print(np.abs(cpuloss - gpuloss))
# print(np.linalg.norm(cpuderivmat-gpuderivmat))

In [73]:
def objgrad(thetaR):
    theta = thetaR[:(2*bigJ+1)] + 1j*thetaR[(2*bigJ+1):]
    obj, grad = MMDadjGPU(theta)
    return obj, np.concatenate([np.real(grad), -np.imag(grad)])

In [36]:
theta0r = np.zeros(2*(2*bigJ+1))

res = scipy.optimize.minimize(fun=objgrad, x0=theta0r, jac=True, method='trust-constr', 
                              options={'gtol': 1e-9, 'xtol': 1e-9, 'verbose': 2})

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  |
|-------|-------|-------|-------------|----------|----------|----------|
|   1   |   1   |   0   | +8.3093e+00 | 1.00e+00 | 3.78e+00 | 0.00e+00 |
|   2   |   2   |   1   | +8.3093e+00 | 2.34e-01 | 3.78e+00 | 0.00e+00 |
|   3   |   3   |   3   | +8.3093e+00 | 2.34e-02 | 3.78e+00 | 0.00e+00 |
|   4   |   4   |   4   | +8.3093e+00 | 1.17e-02 | 3.78e+00 | 0.00e+00 |
|   5   |   5   |   5   | +8.2213e+00 | 2.34e-02 | 3.54e+00 | 0.00e+00 |
|   6   |   6   |   7   | +7.9617e+00 | 1.64e-01 | 3.09e+00 | 0.00e+00 |
|   7   |   7   |  10   | +7.8304e+00 | 1.64e-01 | 2.63e+00 | 0.00e+00 |
|   8   |   8   |  14   | +7.4744e+00 | 4.80e-01 | 1.74e+00 | 0.00e+00 |
|   9   |   9   |  18   | +7.4744e+00 | 4.80e-02 | 1.74e+00 | 0.00e+00 |
|  10   |  10   |  21   | +7.4744e+00 | 2.40e-02 | 1.74e+00 | 0.00e+00 |
|  11   |  11   |  23   | +7.4311e+00 | 4.80e-02 | 1.01e+00 | 0.00e+00 |
|  12   |  12   |  27   | +7.4009e+00 | 1.35e-01 | 

In [18]:
objgrad(res.x)[0]

7.127728698797097

In [41]:
np.savez('trustresdblwell.npz',res.x)

In [20]:
# myres = np.array([-1.25946484e-03,  4.74873183e-03,  1.53364412e-02, -1.17326172e-02, 
#                   6.43005703e-03, -1.17341546e-02,  1.53372035e-02,  4.74964768e-03, 
#                   -1.26088174e-03, -1.79119114e-02, -3.51999786e-03,  4.69418203e-01, 
#                   7.43126275e-03, -1.14321699e-07, -7.43141730e-03, -4.69418964e-01, 3.52053755e-03,  1.79124299e-02])

In [86]:
myres = np.load('trustresdblwell.npz')['arr_0'] # res.x
bigJ = 16
myresC = myres[:(2*bigJ+1)] + 1j*myres[(2*bigJ+1):]
# np.mean(np.square(np.abs(myresC - truetheta)))

In [89]:
# now construct f on a plotting grid
nplot = 129
# xplot = np.linspace(-2*np.pi,2*np.pi, nplot)
xplot = np.linspace(-1.5,1.5, nplot)
f = np.zeros((nplot), dtype=np.complex128)
for j in range(2*bigJ+1):
    f += myresC[j]*np.exp(1j*xplot*modes[j])

font = {'family' : 'serif',
    'weight' : 'regular',
    'size'   : 20}

matplotlib.use('Agg')
# np.sin(xplot) xplot - xplot**3
matplotlib.rc('font', **font)
plt.figure(figsize=(6,6))
plt.plot(xplot, np.real(f), color='red',label='estimate')
plt.plot(xplot, xplot - xplot**3, color='black', label='truth')
plt.xlabel('x')
#plt.gca().yaxis.set_label_coords(1.1,1.0)
# plt.yticks(ticks=np.array([-1.5,-1.,-0.5,0.0,0.5,1.,1.5]),labels=["-1.5","-1.0","-0.5","","0.5","1.0","1.5"])
# plt.gca().yaxis.tick_right()
plt.title('true and estimated drifts $f$')
plt.legend(loc='upper center',prop={'size': 14},frameon=True, bbox_to_anchor=(0.831, 1.02))
plt.grid()
plt.show()
plt.savefig('dblwell1d.pdf')
# plt.savefig('sin1d.pdf')
plt.close()

In [23]:
# truetheta

In [94]:
thetaspace = np.linspace(0.0, 1.0, 100)
lossspace = np.zeros(100)
for ii in range(100):
    thistheta = np.zeros(2*bigJ+1, dtype=np.complex128)
    thistheta[bigJ+bigL] = -1j * thetaspace[ii]
    thistheta[bigJ-bigL] = 1j * thetaspace[ii]
    lossspace[ii] = MMDlossGPU(thistheta)

In [95]:
font = {'family' : 'serif',
    'weight' : 'regular',
    'size'   : 20}

matplotlib.use('Agg')
# np.sin(xplot) xplot - xplot**3
matplotlib.rc('font', **font)
plt.figure(figsize=(6,6))
plt.plot(thetaspace, lossspace, color='red')
plt.axvline(x=thetaspace[np.argmin(lossspace)], color='blue')
plt.axvline(x=0.5, color='black')
plt.xlabel('$ \\theta $')
plt.title('MMD Loss($ \\theta $)')
plt.savefig('sin1dmmdloss.pdf',bbox_inches='tight')
plt.close()

In [26]:
# truef = np.zeros(nplot, dtype=np.complex128)
# for j in range(2*bigJ+1):
#     truef += truetheta[j] * np.exp((j - bigJ)*1j*xplot/2)

# plt.plot(xplot, truef)
# plt.savefig('randomtestfig.pdf')
# plt.close()

In [41]:
myresC

array([ 0.01021105-1.64430735e-02j, -0.01074967-4.52742517e-04j,
        0.01435173+4.88982669e-01j, -0.01890702-1.54525160e-02j,
        0.00757787-9.25973354e-04j, -0.0188658 +1.64661631e-02j,
        0.01328547-4.88638344e-01j, -0.0094562 -8.14386928e-04j,
        0.00789035+1.72425304e-02j])

In [33]:
thetaspace[np.argmin(lossspace)]

0.47847847847847846

In [43]:
bigJ+bigL

6